In [ ]:
import math
import numpy as np
from dateutil import parser
import os
import random
import pickle
import matplotlib.pyplot as plt
import time
import copy
%matplotlib notebook

In [ ]:
length_fragment = 2000 #длина фрагментов
freq = 20 #частота
fi = [35.307756, 35.312867, 35.311456, 35.304555, 35.299183, 35.307906] #широты каналов
tetta = [140.313762, 140.318017, 140.329999, 140.326213, 140.313643, 140.307005] #долготы каналов

In [ ]:
class ParseFiles:
    FREQ_INFRA = 20
    FREQ_WING = 20
    COUNT = 72000 #Стандартная длина фрагмента. Пока исходим из ее постоянства

    def __init__(self, name):
        self.name = name

    def ParseAll(self):
        self.coefficientWing = 0
        self.stations = []
        fileText = []
        path = "/Volumes/ELEMENTS/base"
        stations = os.listdir(path)
        for station in stations:
            if (station.find("DS") != -1 or station[0] == '.'):
                continue
            years = os.listdir(path + "/" + station)
            st = []
            for year in years:
                if (year.find("DS") != -1 or year[0] == '.'):
                    continue
                files = os.listdir(path + "/" + station + "/" + year)

                for i in range(len(files)):
                    if(files[i].find("DS") != -1 or files[i].find("icloud") != -1 or files[i][0] == '.'):
                        continue
                        with open(path + "/" + station + "/" + year + "/" + files[i]) as fileInfra:
                            for line in fileInfra:
                                line = line.replace('\n', '')
                                words = line.split(' ')
                                words = filter(None, words)
                                for word in words:
                                    fileText.append(word)
                    a = self.ParseFiles(path + "/" + station + "/" + year + "/" + files[i])
                    if not (a is None):
                        st.append(a)
#                     if a:
#                         st.append(a)

        return st

    def ParseFiles(self, path):
        f = open(path, "r")
        print(path)
        f = f.read()
        f = f.replace('\n', ' ')
        f = f.split(' ')
        fileText = [item for item in f if item != '']

        channelInfra = []
        channelWing = []


        nameLast = "" #для сохранения имени предыдущей части данных
        chaLast = ""
        i = fileText.index("WID2")
        name = ""
        cha = ""
        while True:
            if fileText[i] == 'WID2':
                date = fileText[i + 1]
                time = fileText[i + 2]
                date = parser.parse(date + ' ' + time)
                nameLast = name
                chaLast = cha
                name = fileText[i + 3]
                cha = fileText[i + 4]
                count = int(fileText[i + 6])
#                 проверка на случай повторяющихся каналов, бывали случаи, когда канал разбивался на две подрядыдущие порции
#                 возможно такая же проверка нужна будет для ветра
                if (name == nameLast) and (chaLast == 'BDF') and (fileText[i + 4] == 'BDF'):
                    print(name)
                    channelInfra[-1] = channelInfra[-1] + fileText[i+21:i+21+count]
                    i += 21 + count
                    fileText = fileText[i:]
                    try:
                        i = fileText.index("WID2")
                    except:
                        break
                    continue
                if fileText[i + 4] == 'BDF':
                    latitude = float(fileText[i + 15])
                    longitude = float(fileText[i + 16])
                    self.FREQ_INFRA = float(fileText[i + 7])
                    coefficientInfra = float(fileText[i + 8])
                    channelInfra.append(fileText[i+21:i+21+count])
                    
                    i += 21 + count
                    fileText = fileText[i:]
                    try:
                        i = fileText.index("WID2")
                    except:
                        break
                    continue
                if fileText[i + 4] == 'BWS' or fileText[i + 4] == 'LWS':
                    self.coefficientWing = float(fileText[i + 8])
                    channelWing = fileText[i+21:i+21+count]
                    self.FREQ_WING = float(fileText[i + 7])
                    i += 21 + count
                    fileText = fileText[i:]
                    try:
                        i = fileText.index("WID2")
                    except:
                        break
                    continue
                if fileText[i + 4] == 'LDA' or fileText[i + 4] == 'LKO' or fileText[i + 4] == 'LWD'\
                or fileText[i + 4] == 'BDA' or fileText[i + 4] == 'BKO' or fileText[i + 4] == 'BWD':
                    i += 21 + count
                    fileText = fileText[i:]
                    try:
                        i = fileText.index("WID2")
                    except:
                        break
                    continue


        #проверка, если на каком-то из каналов запись будет не полная, то канал заполняется до полного своими же значениями
        for chan in channelInfra:
            if len(chan) < self.COUNT:
                while 1:
                    if self.COUNT - len(chan) > len(chan):
                        chan += chan[0: len(chan)]
                    else:
                        chan += chan[0: self.COUNT - len(chan)]
                        break

        if len(channelWing) != 72000:
            return None
        
        channelInfra.append(channelWing)
        print(len(channelWing))
        
        channelInfra = np.array(channelInfra, dtype='int')*coefficientInfra
        
        if channelInfra.shape != (7, 72000):
            print(channelInfra.shape, "error")
            return None
        else:
            print(channelInfra.shape, 'success')
        channelWing = np.array(channelWing, dtype='int')*self.coefficientWing
        print(channelWing.shape)

        return channelInfra

In [ ]:
def delay(azimut):
    f0 = fi[0]
    t0 = tetta[0]
    rz = 6371000
    c = 330
    a = math.radians(30)
    azimut = math.radians(azimut)
    t_channel = []
    for i in range(len(fi)):
        x = (math.radians(tetta[i]) - math.radians(t0))*rz*math.cos(math.radians(f0))
        y = (math.radians(fi[i]) - math.radians(f0))*rz
        r = math.sqrt(x**2 + y**2)
        if x != 0 and y != 0:         
            
            if x > 0 and y > 0:
                b = math.atan(abs(y/x))
            if x < 0 and y > 0:
                b = math.pi - math.atan(abs(y/x))
            if x < 0 and y < 0:
                b = math.pi + math.atan(abs(y/x))
            if x > 0 and y < 0:
                b = 2*math.pi - math.atan(abs(y/x))
                
        elif x == 0 and y > 0:
            b = math.pi/2
        elif x == 0 and y < 0:
            b = 3*math.pi/2
        elif x > 0 and y == 0:
            b = 0
        elif x < 0 and y == 0:
            b = math.pi
        else:
            b = 0

            
        V = c/math.cos(a)
        time = (r/V)*math.cos(math.radians(90)-azimut-b)
        t_channel.append(time)
    m = min(t_channel)
    for i in range(len(t_channel)):
        t_channel[i] += abs(m)
    return t_channel

In [ ]:
def CalcAmplitude(c, q):
    R = 1238
    T = int((10 + 5*math.log10(q))*math.pow(q,1/3))
    freq = 20
    OMEGA = (0.2/math.pow(q,1/3))*2*math.pi
    A0 = math.pow(10, 3.25 + 0.54*math.log10(q) - 1.33*math.log10(R))
    Signals = []
    count = T*freq
    for i in range(c):
        A = math.pow(10, math.log10(A0) + math.sqrt(0.0357)*np.random.normal(0, 1))
        A2w = 0.25*A
        Aw = 0.5*A
        Aw2 = 0.25*A
        w = OMEGA
        w2w = 2*OMEGA
        ww2 = OMEGA/2
        Signal = []
        
        for j in range (count):
            Signal.append(A2w*math.sin(w2w*j/freq) + Aw*math.sin(w*j/freq) + Aw2*math.sin(ww2*j/freq))
        Signals.append(Signal)
    return np.array(Signals)

In [ ]:
def dft(sig, q):
    T = int((10 + 5*math.log10(q))*math.pow(q,1/3))
    mi = math.ceil(T/(math.pow(q,1/3)*10))
    ma = math.floor((T * 0.6)/(math.pow(q,1/3)))
    const = T * freq / 2
    res = np.fft.rfft(sig)[mi:ma]
    res /= const
    return res

In [ ]:
def correlation(s1, s2):
    a11, a12, a13 = 0, 0, 0
    corr_res = 0
    for i in range(len(s1)):
        a11 += (s1[i].real*s2[i].real +s1[i].imag*s2[i].imag)
        a12 += (s1[i].real**2 + s1[i].imag**2)
        a13 += (s2[i].real**2 + s2[i].imag**2)
    corr_res = a11/math.sqrt(a12*a13)
    return corr_res

In [ ]:
def corr_all(signal_mas):
    corr_mas = []
    for i in range(len(signal_mas)):
        for j in range(len(signal_mas)):
            if i >= j:
                continue
            corr_mas.append(correlation(signal_mas[i], signal_mas[j]))
    res_corr = 0
    for c in corr_mas:
        res_corr += c
    res_corr /= len(corr_mas)
    return res_corr

# Парсинг и обработка

In [ ]:
parser_files = ParseFiles('I30')
df = parser_files.ParseAll()

In [ ]:
len(df)

In [ ]:
dataset_fragm = []
# result_dataset_param = []
for flow in df:
    for i in range(int(len(flow[0])/length_fragment)):
        fragment_noise = flow[:,i*length_fragment:(i+1)*length_fragment]
        dataset_fragm.append(fragment_noise)
#         result_dataset_param.append(np.array([0, None, 0, 0]))
#         result_dataset.append(fragment_noise, 0, None, 0, 0)

In [ ]:
sig = CalcAmplitude(1, 10)
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)
ax.plot(sig[0])
j = 0
time_delays = delay(160)
channels = []
temp_data = copy.deepcopy(dataset_fragm[85])
for fragment in temp_data[:-1]:
    fragment[int(freq*time_delays[j]):int(freq*time_delays[j])+len(sig[0])] += sig[0]
    j += 1
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)
i = 1
for channel in temp_data:
    if i==7:
        ax.plot(channel, label = 'W')
    else:
        ax.plot(channel, label = 'H' + str(i))
    i+=1
ax.legend()
plt.show()
for c in range(1):
    i = 1
    fragment = dataset_fragm[85]
    fig = plt.figure(figsize=(10,6))
    ax = fig.add_subplot(111)
    for channel in fragment:
        if i==7:
            ax.plot(channel, label = 'W')
        else:
            ax.plot(channel, label = 'H' + str(i))
        i+=1
    ax.legend()
    plt.show()
#     time.sleep(0.5)

In [ ]:
len(dataset_fragm)

In [ ]:
random.shuffle(dataset_fragm)

In [ ]:
noise_fragm = []
noise_param = []
i = 0
for a in range(0,360,10):
    time_delays = delay(a)
    max_delay = max(time_delays) 
    for q in range(1, 100, 4):
        noise_fragments = copy.deepcopy(dataset_fragm[i*47:(i+1)*47])
        T = int((10 + 5*math.log10(q))*math.pow(q,1/3))
        i += 1
        print(i, a, q)
        for noise_fragment in noise_fragments:
            step = int(T/3)
            corrs = []
            k = 0
            while int(freq*max_delay + T + step*k) < length_fragment:
                channels = []
                j = 0
                for fragment in noise_fragment[:-1]:
                    channels.append(dft(fragment[int(freq*time_delays[j]) + step*k:int(freq*time_delays[j])+T + step*k], q))
                    j += 1
                k += 1
                corr = corr_all(channels)
                corrs.append(corr)
            noise_fragm.append(noise_fragment)
            noise_param.append(np.array([q, a, max(corrs), 0]))
#             print(k)

In [ ]:
len(noise_fragm)

In [ ]:
signals_fragm = []
signals_param = []
i = 0
for a in range(0,360,10):
    for q in range(1, 100, 4):
        signal_fragments = copy.deepcopy(dataset_fragm[i*47:(i+1)*47])
        i += 1
        print(i, a, q)
        for signal_fragment in signal_fragments:
            sig = CalcAmplitude(1, q)
            j = 0
            time_delays = delay(a)
            channels = []
            for fragment in signal_fragment[:-1]:
                fragment[int(freq*time_delays[j]):int(freq*time_delays[j])+len(sig[0])] += sig[0]
                channels.append(dft(fragment[int(freq*time_delays[j]):int(freq*time_delays[j])+len(sig[0])], q))
                j += 1
            corr = corr_all(channels)
            signals_fragm.append(signal_fragment)
            signals_param.append(np.array([q, a, corr, 1]))

In [ ]:
len(signals_fragm)

In [ ]:
result_dataset_fragm = signals_fragm + noise_fragm
result_dataset_param = signals_param + noise_param

In [ ]:
len(result_dataset_fragm)

In [ ]:
result_dataset_fragm = np.array(result_dataset_fragm).astype('float32')
result_dataset_param = np.array(result_dataset_param).astype('float32')

In [ ]:
print(result_dataset_fragm.shape)
print(result_dataset_param.shape)

In [ ]:
p = np.random.permutation(len(result_dataset_fragm))
result_dataset_fragm = result_dataset_fragm[p]
result_dataset_param = result_dataset_param[p]

In [ ]:
for c in range(75, 80):
    i = 1
    fragment = result_dataset_fragm[c]
    paramet = result_dataset_param[c]
    print(paramet[0], paramet[1], paramet[2], paramet[3])
    fig = plt.figure(figsize=(10,6))
    ax = fig.add_subplot(111)
    for channel in fragment:
        if i==7:
            ax.plot(channel, label = 'W')
        else:
            ax.plot(channel, label = 'H' + str(i))
        i+=1
    ax.legend()
    plt.show()
#     time.sleep(0.5)

In [ ]:
with open('/Users/igorfrolkin/Desktop/data_param_various_qa_3.pkl', 'wb+') as f:
    np.save(f, result_dataset_param)

In [ ]:
y_real = result_dataset_param[:, 3]

In [ ]:
y_proba = result_dataset_param[:, 2]

In [ ]:
y_proba = abs(y_proba)

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr_rf, tpr_rf, _ = roc_curve(y_real, y_proba)
roc_auc = auc(fpr_rf, tpr_rf)

In [ ]:
roc_auc

In [ ]:
ro = -1
count = y_real.shape[0]
x = []
y = []
while ro <= 1:
    y1 = np.argwhere((y_proba > ro)&(y_real == 1)).shape[0]/count*2
    x1 = np.argwhere((y_proba > ro)&(y_real == 0)).shape[0]/count*2
    x.append(x1)
    y.append(y1)
    ro += 0.001

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.plot([0, 1], [0, 1], 'k--')
ax.plot(x,y)

In [ ]:
fig = plt.figure(figsize=(18,9))
ax = fig.add_subplot(111)
ax.hist(y_proba[np.argwhere(y_real == 1)], bins=[i/100 for i in range(-100,101,1)], color='red', alpha=0.7)
ax.hist(y_proba[np.argwhere(y_real == 0)], bins=[i/100 for i in range(-100,101,1)], color='blue', alpha=0.7)